In [ ]:
import pandas as pd
import polars as pl
import numpy as np
import random
import sys
sys.path.insert(1, '../rtsvg')
from rtsvg import *
rt = RACETrack()

In [ ]:
xs, ys, ws  = [], [], []
for samples in range(30000):
    xs.append(random.random())
    ys.append(random.random())
    ws.append(random.random())
df    = pd.DataFrame({'x': xs, 'y': ys, 'w':ws})
df_pl = pl.DataFrame(df)
rt.xy(df, x_field='x', y_field='y', color_by='ws', dot_size='small', color_magnitude='stretch')

In [ ]:
import time
samples  = 3
time_sum = 0.0
print('== pandas ==')
for i in range(samples):
    t0 = time.time()
    xy_obj = rt.xy(df, x_field='x', y_field='y', w=256, h=256)
    xy_obj._repr_svg_()
    t1 = time.time()
    # xy_obj.performanceInformation()
    time_sum += (t1 - t0)
pd_time = time_sum / samples
time_sum = 0.0
print()
print('== polars ==')
for i in range(samples):
    t0 = time.time()
    xy_obj = rt.xy(df_pl, x_field='x', y_field='y', w=256, h=256)
    xy_obj._repr_svg_()
    t1 = time.time()
    # xy_obj.performanceInformation()
    time_sum += (t1 - t0)
pl_time = time_sum / samples
print()
print(f'pandas {pd_time} || polars {pl_time} || {pl_time/pd_time}')

In [ ]:
sz = 1000000

ranges, polars_t, pandas_t = [], [], []

for _range_ in range(20,1025,200):
    xs, ys = [], []
    for i in range(sz):
        xs.append(random.randint(1,_range_)), ys.append(random.randint(1,_range_))
    df_pd = pd.DataFrame({'x':xs, 'y':ys})
    df_pl = pl.DataFrame(df_pd)

    t0 = time.time()
    gb = df_pd.groupby(['x','y'])
    my_sum = 0
    for k,k_df in gb:
        my_sum += len(k)
    t1 = time.time()

    t_pd = t1 - t0

    t0 = time.time()
    gb = df_pl.group_by(['x','y'])
    my_sum = 0
    for k,k_df in gb:
        my_sum += len(k)
    t1 = time.time()

    t_pl = t1 - t0
    # print(f'{t_pd:0.4f} | {t_pl:0.4f}')
    ranges.append(_range_), polars_t.append(t_pl), pandas_t.append(t_pd)

In [ ]:
xs, ys, types = [], [], []
for i in range(len(ranges)):
    xs.append(ranges[i])
    ys.append(pandas_t[i])
    types.append('pandas')
    xs.append(ranges[i])
    ys.append(polars_t[i])
    types.append('polars')
rt.co_mgr.str_to_color_lu['pandas'] = '#ffa0a0' # red-ish
rt.co_mgr.str_to_color_lu['polars'] = '#a0a0ff' # blue-ish
rt.xy(pd.DataFrame({'range':xs, 'time':ys, 'lib':types}), x_field='range', y_field='time', line_groupby_field='lib', line_groupby_w=3, color_by='lib', dot_size=None, w=512, h=384)